In [32]:
import numpy as np
import pandas as pd


 
# month 두자리를 string으로 입력
def get_taker_maker_ratio(month):

    # 파일 경로
    file_path = "BTCUSDT-trades-2023-{}.csv".format(month)

    # 로드할 열의 리스트
    selected_columns = ["quote_qty", "time", "is_buyer_maker"]
    df = pd.read_csv(file_path, usecols=selected_columns)

    # 'time' 열을 시간 단위로 변환 (현재는 밀리초 단위)
    df['time'] = pd.to_datetime(df['time'], unit='ms')

    # 1시간 간격으로 그룹화하고 is_buyer_maker에 따른 quote_qty 합 계산
    result = df.groupby([pd.Grouper(key='time', freq='1H'), 'is_buyer_maker']).agg({'quote_qty': 'sum'}).reset_index()


    # 'taker/maker_ratio' 칼럼 추가
    result['taker/maker_ratio'] = result.groupby('time')['quote_qty'].transform(lambda x: x.iloc[0] / x.iloc[1])

    # 'taker/maker_diff' 칼럼 추가
    result['taker-maker_diff'] = result.groupby('time')['quote_qty'].transform(lambda x: x.iloc[0] - x.iloc[1])

    # 짝수 행 삭제(중복처리)
    result = result[result.index % 2 != 0].reset_index(drop=True)

    
    # 필요한 열만 선택
    result = result[['time', 'taker/maker_ratio', 'taker-maker_diff']]

    # 데이터프레임 출력
    result.to_csv("taker_maker_ratio{}.csv".format(month))
    return None



In [34]:
for i in range(1,14):
    month = '{:02d}'.format(i)
    get_taker_maker_ratio(month)

C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\2779920026.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result = df.groupby([pd.Grouper(key='time', freq='1H'), 'is_buyer_maker']).agg({'quote_qty': 'sum'}).reset_index()
C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\2779920026.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result = df.groupby([pd.Grouper(key='time', freq='1H'), 'is_buyer_maker']).agg({'quote_qty': 'sum'}).reset_index()
C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\2779920026.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result = df.groupby([pd.Grouper(key='time', freq='1H'), 'is_buyer_maker']).agg({'quote_qty': 'sum'}).reset_index()
C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\2779920026.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please us

In [35]:
# 빈 DataFrame을 생성
combined_data = pd.DataFrame()

for i in range(1,14):
    month = '{:02d}'.format(i)
    df=pd.read_csv("taker_maker_ratio{}.csv".format(month))
    combined_data = pd.concat([combined_data, df], ignore_index=True)

In [36]:
combined_data

,Unnamed: 0,time,taker/maker_ratio,taker-maker_diff
0,0,2023-01-01 00:00:00,0.895108,-4.921784e+06
1,1,2023-01-01 01:00:00,1.514577,1.086594e+07
2,2,2023-01-01 02:00:00,0.930625,-1.426735e+06
3,3,2023-01-01 03:00:00,0.801744,-5.845137e+06
4,4,2023-01-01 04:00:00,1.009731,2.520080e+05
...,...,...,...,...
9379,643,2024-01-27 19:00:00,1.363879,5.008726e+07
9380,644,2024-01-27 20:00:00,1.180081,1.875999e+07
9381,645,2024-01-27 21:00:00,1.183708,2.074670e+07
9382,646,2024-01-27 22:00:00,0.885899,-1.343574e+07


In [38]:
combined_data['taker-maker_diffsum']=combined_data['taker-maker_diff'].cumsum()
combined_data=combined_data[['time','taker/maker_ratio','taker-maker_diffsum']]

In [39]:
combined_data.to_csv("taker_maker_ratio_all.csv")

In [40]:
import pandas as pd
import numpy as np
df2=pd.read_csv("taker_maker_ratio_all.csv")

In [41]:
df2['time'] = pd.to_datetime(df2['time'])  # 문자열을 datetime 형식으로 변환
df2.set_index('time', inplace=True)  # time을 인덱스로 설정

# 전체 시간 범위 생성 (1시간 간격)
start_time = df2.index.min()
end_time = df2.index.max()
full_time_range = pd.date_range(start=start_time, end=end_time, freq='1H')

# 빠진 시간대 찾기
missing_time = full_time_range.difference(df2.index)
print("빠진 시간대:")
print(missing_time)

빠진 시간대:
DatetimeIndex(['2023-01-23 21:00:00', '2023-01-23 22:00:00',
               '2023-01-23 23:00:00', '2023-01-24 21:00:00',
               '2023-01-24 22:00:00', '2023-01-24 23:00:00',
               '2023-01-25 16:00:00', '2023-01-25 17:00:00',
               '2023-01-25 18:00:00', '2023-01-25 19:00:00',
               '2023-01-25 20:00:00', '2023-01-25 21:00:00',
               '2023-01-25 22:00:00', '2023-01-25 23:00:00',
               '2023-01-26 22:00:00', '2023-01-26 23:00:00',
               '2023-03-14 23:00:00', '2023-03-15 22:00:00',
               '2023-03-15 23:00:00', '2023-11-21 03:00:00',
               '2023-11-21 04:00:00', '2023-11-21 05:00:00',
               '2023-11-21 06:00:00', '2023-11-21 07:00:00'],
              dtype='datetime64[ns]', freq=None)


C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\3284806919.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_time_range = pd.date_range(start=start_time, end=end_time, freq='1H')


In [42]:
df2=pd.read_csv("taker_maker_ratio_all.csv")

In [43]:
missing_df = pd.DataFrame(missing_time,columns=['time'])
missing_df['taker/maker_ratio'] = np.nan
missing_df['time'] = pd.to_datetime(missing_df['time'])

# 시간을 기준으로 두 데이터프레임 합치기
merged_df = pd.concat([df2, missing_df], ignore_index=True, axis=0)
merged_df['time'] = pd.to_datetime(merged_df['time'])

# 시간을 기준으로 정렬
merged_df.sort_values(by='time', inplace=True)

# 인덱스 재설정
merged_df.reset_index(drop=True, inplace=True)

# NaN 값을 1로 채우기(거래가 1시간동안 발생하지 않음, 바이낸스 점검 등 이슈로 인한 것으로 추측, 데이터 부재)
merged_df['taker/maker_ratio'].fillna(value=1, inplace=True)

merged_df

C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\1436147825.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['taker/maker_ratio'].fillna(value=1, inplace=True)


,Unnamed: 0,time,taker/maker_ratio,taker-maker_diffsum
0,0.0,2023-01-01 00:00:00,0.895108,-4.921784e+06
1,1.0,2023-01-01 01:00:00,1.514577,5.944155e+06
2,2.0,2023-01-01 02:00:00,0.930625,4.517420e+06
3,3.0,2023-01-01 03:00:00,0.801744,-1.327717e+06
4,4.0,2023-01-01 04:00:00,1.009731,-1.075709e+06
...,...,...,...,...
9403,9379.0,2024-01-27 19:00:00,1.363879,-1.798520e+10
9404,9380.0,2024-01-27 20:00:00,1.180081,-1.796644e+10
9405,9381.0,2024-01-27 21:00:00,1.183708,-1.794570e+10
9406,9382.0,2024-01-27 22:00:00,0.885899,-1.795913e+10


In [45]:
merged_df.to_csv("full_takermaker.csv")

In [46]:
ohlcvdata=pd.read_csv('df.csv')

In [49]:
ohlcvdata=ohlcvdata.iloc[:,1:]

###전처리###

In [50]:
ohlcvdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9408 entries, 0 to 9407
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   time        9408 non-null   object 
 1   Open        9384 non-null   float64
 2   High        9384 non-null   float64
 3   Low         9384 non-null   float64
 4   Close       9384 non-null   float64
 5   Volume      9408 non-null   float64
 6   returns     9407 non-null   float64
 7   volatility  9388 non-null   float64
dtypes: float64(7), object(1)
memory usage: 588.1+ KB


In [51]:
#Nan 예시
ohlcvdata.iloc[596]

time          2023-01-25 20:00:00
Open                          NaN
High                          NaN
Low                           NaN
Close                         NaN
Volume                        0.0
returns                       0.0
volatility               0.003256
Name: 596, dtype: object

In [52]:
#Open 열이 Nan을 가지면 이전시점의 종가로 대체한다.
for i in range(100):
    ohlcvdata['Open'].fillna(ohlcvdata['Close'].shift(i), inplace=True)
    # 만약 NaN이 아닌 값을 찾았다면 루프 중단
    if not ohlcvdata['Open'].isna().any():
        break


C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\1153217793.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ohlcvdata['Open'].fillna(ohlcvdata['Close'].shift(i), inplace=True)


In [53]:
# high,low,close 열에 대해 NaN 값을 해당 시점의 'Open' 값으로 채우기
ohlcvdata.fillna(value={'High': ohlcvdata['Open'], 'Low': ohlcvdata['Open'], 'Close': ohlcvdata['Open']}, inplace=True)

In [54]:
#실제로 데이터 없는 1월 24일 데이터 변환예시
ohlcvdata.iloc[596]

time          2023-01-25 20:00:00
Open                      22537.7
High                      22537.7
Low                       22537.7
Close                     22537.7
Volume                        0.0
returns                       0.0
volatility               0.003256
Name: 596, dtype: object

In [65]:
def calculate_volatility(data, window=20):
    """
    Calculate the rolling volatility using the standard deviation of returns.
    :param data: DataFrame with OHLCV data
    :param window: The number of periods to use for calculating the standard deviation
    :return: DataFrame with the volatility values
    """

    # Calculate daily returns
    data['returns'] = data['Close'].pct_change()

    # Calculate the rolling standard deviation of returns
    data['volatility'] = data['returns'].rolling(window=window).std()

    return data

df=calculate_volatility(ohlcvdata)

In [66]:
#년단위 재계산 통해 nan 값 : return 1개, volatility 20개로 줄임.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9408 entries, 0 to 9407
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   time        9408 non-null   object 
 1   Open        9408 non-null   float64
 2   High        9408 non-null   float64
 3   Low         9408 non-null   float64
 4   Close       9408 non-null   float64
 5   Volume      9408 non-null   float64
 6   returns     9407 non-null   float64
 7   volatility  9388 non-null   float64
dtypes: float64(7), object(1)
memory usage: 588.1+ KB


In [67]:
# 두개 열의 NaN 값을 각 행의 평균값으로 대체
returns_mean = df['returns'].mean()
df['returns'].fillna(returns_mean, inplace=True)

volatility_mean = df['volatility'].mean()
df['volatility'].fillna(volatility_mean, inplace=True)

C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\2741563265.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['returns'].fillna(returns_mean, inplace=True)
C:\Users\Taewan\AppData\Local\Temp\ipykernel_8480\2741563265.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [69]:
#첫행 filled 예
df.iloc[0]

time          2023-01-01 00:00:00
Open                      16537.5
High                      16540.9
Low                       16504.0
Close                     16527.0
Volume                   5381.399
returns                   0.00011
volatility               0.003982
Name: 0, dtype: object

##taker/maker데이터 합치기##

In [70]:
df_ratio=pd.read_csv('full_takermaker.csv')

In [72]:
df['time']=df_ratio['time']
df['taker/maker_ratio']=df_ratio['taker/maker_ratio']
df['taker-maker_diffsum']=df_ratio['taker-maker_diffsum']

# 'time' 열의 위치를 0번째로 이동
df.insert(0, 'time', df.pop('time'))
df

,time,Open,High,Low,Close,Volume,returns,volatility,taker/maker_ratio,taker-maker_diffsum
0,2023-01-01 00:00:00,16537.5,16540.9,16504.0,16527.0,5381.399,0.000110,0.003982,0.895108,-4.921784e+06
1,2023-01-01 01:00:00,16527.1,16554.3,16524.1,16550.4,3210.826,0.001416,0.003982,1.514577,5.944155e+06
2,2023-01-01 02:00:00,16550.5,16557.1,16534.8,16542.4,2399.668,-0.000483,0.003982,0.930625,4.517420e+06
3,2023-01-01 03:00:00,16542.5,16542.5,16515.0,16529.3,3214.480,-0.000792,0.003982,0.801744,-1.327717e+06
4,2023-01-01 04:00:00,16529.2,16530.4,16508.8,16517.8,3150.954,-0.000696,0.003982,1.009731,-1.075709e+06
...,...,...,...,...,...,...,...,...,...,...
9403,2024-01-27 19:00:00,41785.0,41977.0,41775.1,41896.4,7765.565,0.002668,0.002019,1.363879,-1.798520e+10
9404,2024-01-27 20:00:00,41896.5,42070.0,41896.4,42049.6,5408.243,0.003657,0.002154,1.180081,-1.796644e+10
9405,2024-01-27 21:00:00,42049.6,42165.6,42001.8,42137.8,5859.277,0.002098,0.002189,1.183708,-1.794570e+10
9406,2024-01-27 22:00:00,42137.8,42187.1,42057.6,42135.3,5271.863,-0.000059,0.002191,0.885899,-1.795913e+10


In [73]:
df.to_csv("ohlcv_baseline.csv")

In [71]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
